In [1]:
# Dashboard datascience
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

import dash_bootstrap_components as dbc


In [38]:
df = pd.read_csv('../data/clean_game_data.csv', index_col=0)
cleandf = df.select_dtypes(include='number').drop(columns=['Year', 'Game ID']).drop(columns=['Game Duration'])

supdf = df[df['Role'] == 'SUPPORT']
nonsupdf = df[df['Role'] != 'SUPPORT']

elo_df = pd.read_csv('../data/elo_evolution.csv')

elo_df['Game ID'] = range(len(elo_df))
elo_df = elo_df.set_index('Game ID')

top_elo = elo_df.iloc[-1].sort_values(ascending=False).head(10)

top_elo_df = elo_df[top_elo.index]


In [45]:



external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']



app = Dash(__name__, external_stylesheets=[dbc.themes.PULSE])

app.layout = dbc.Container([
    dbc.Row([
        html.H1("LOL Esports Analyse"),
        html.Hr(),
    ]),
    
    dbc.Row([
        dcc.RadioItems(options=['Mono-variate analysis', 'Bivariate analysis'], value='Mono-variate analysis', id='control_radio_items'),
        html.Hr()
    ]),
    
    dbc.Row([html.Div(id='EDA-analysis-Container')]),
    
    
    dbc.Row([
        dcc.Graph(id='ELO', figure=px.line(elo_df, title='ELO evolution', labels={'index':'Game ID', 'value':'ELO'}))
    ])
    
    
    
    
], fluid=True)

@callback(
    Output('graph-content-universal', 'figure'),
    Input('dropdown-selection-universal', 'value')
)
def update_graph(value):
    return px.histogram(cleandf, x=value)

@callback(
    Output('graph-content-sup-nonsup', 'figure'),
    Input('dropdown-selection-sup-nonsup', 'value')
)
def update_graph(value):
    fig = px.histogram(supdf[value], x=value, title=f'{value} Distribution Support vs Non-support',  color_discrete_sequence=['blue'], opacity=0.5, labels={value:f'{value} for support'})
    fig.add_trace(px.histogram(nonsupdf[value], x=value, color_discrete_sequence=['purple'], labels={value:f'{value} for non support'}, opacity = 0.5).data[0])
    fig.update_layout(barmode='overlay', xaxis_title=value, yaxis_title='Count', showlegend=True)
    return fig

@callback(
    Output('bi-var', 'figure'),
    Input('dropdown-selection-first-var', 'value'),
    Input('dropdown-selection-second-var', 'value'),
)

def update_graph(x, y):
    return px.scatter(cleandf, x=x, y=y, title=f'{x} vs {y}')

@callback(
    Output('EDA-analysis-Container', 'children'),
    Input('control_radio_items', 'value')
)
def update_EDA_analysis(value):
    if value == 'Mono-variate analysis':
        return dbc.Row([
        dbc.Col([
            html.H3('Univariate analysis'),
            dcc.Dropdown(cleandf.columns, 'Kills', id='dropdown-selection-universal'),
            dcc.Graph(id='graph-content-universal')
        ], width=5),
        
        dbc.Col([], width=2),
        
        dbc.Col([
            html.H3('Sup vs. NonSup analysis'),
            dcc.Dropdown(cleandf.columns, 'Kills', id='dropdown-selection-sup-nonsup'),
            dcc.Graph(id='graph-content-sup-nonsup')
        ], width=5),
        html.Hr()
    ])
    
    elif value == 'Bivariate analysis':
        return (dbc.Row([
                html.H3('Bivariate analysis'),
            ]),
    
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(cleandf.columns, 'Golds', id='dropdown-selection-first-var'),
            ],width=3),
            
            dbc.Col([
                dcc.Dropdown(cleandf.columns, 'Kills', id='dropdown-selection-second-var'),
            ],width=3),
            
        ], justify='center'),
        
        dbc.Row([
            dbc.Col([
                dcc.Graph(id='bi-var')
            ], width=6)
        ],justify='center'))

if __name__ == '__main__':
    app.run(debug=True)